In [23]:
import math,torch,matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from operator import attrgetter
from functools import partial
from copy import copy

from torch import optim
from torch.utils.data import DataLoader, default_collate
import torch.nn.functional as F

from miniai.conv import *

from fastprogress import progress_bar,master_bar

In [24]:
import matplotlib as mpl
import torchvision.transforms.functional as TF
from contextlib import contextmanager
from torch import nn,tensor
from datasets import load_dataset,load_dataset_builder
from miniai.datasets import *
from miniai.conv import *
import logging
from fastcore.test import test_close

In [25]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [26]:
logging.disable(logging.WARNING)

# Learner

In [27]:
x,y = 'image','label'
name = "fashion_mnist"
dsd = load_dataset(name)

  0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
@inplace 
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [29]:
bs = 1024
tds = dsd.with_transform(transformi)

In [45]:
class DataLoaders:
    def __init__(self, *dls): self.train, self.valid = dls[:2]
    
    @classmethod 
    def from_dd(cls, dd, batch_size, as_tuple = True):
        return cls(*[DataLoader(ds, batch_size, num_workers = 4, collate_fn = collate_dict(ds)) for ds in dd.values()])

In [46]:
dls = DataLoaders.from_dd(tds, bs)
dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:10]

(torch.Size([1024, 784]), tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5]))

In [47]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, opt_func = optim.SGD): fc.store_attr()
    
    def one_batch(self):
        self.xb, self.yb = to_device(self.batch)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        with torch.no_grad(): self.calc_stats()
    
    def calc_stats(self):
        acc = (self.preds.argmax(dim=1)==self.yb).float().sum() 
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss * n)
        self.ns.append(n)
    
    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for self.num, self.batch in enumerate(dl): self.one_batch()
        n = len(self.ns)
        print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)
        
    def fit(self, n_epochs):
        self.accs, self.losses, self.ns = [], [], []
        self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            self.one_epoch(False)
    

In [48]:
m,nh = 28*28,50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [49]:
learn = Learner(model, dls, F.cross_entropy, lr=0.2)
learn.fit(1)

0 True 1200.5618379237287 624.9491525423729
0 False 1161.5954483695652 631.0579710144928


## Basic Callbacks Learner

In [76]:
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

In [93]:
def run_cbs(cbs, method_nm):
    for cb in sorted(cbs, key=attrgetter('order')):
        method = getattr(cb, method_nm, None)
        if method is not None: method()

In [94]:
class Callback(): order = 0

In [95]:
class CompletionCB(Callback):
    def before_fit(self): self.count = 0
    def after_batch(self): self.count += 1
    def after_fit(self): print(f'Completed {self.count} batches')

In [96]:
cbs = [CompletionCB()]
run_cbs(cbs, 'before_fit')
run_cbs(cbs, 'after_batch')
run_cbs(cbs, 'after_fit')

Completed 1 batches


In [101]:
class Learner():
    def __init__(self, model, dls, loss_func, lr, cbs, opt_func = optim.SGD):
        fc.store_attr()
        for cb in cbs: cb.learn = self
        
    def one_batch(self):
        self.preds = self.model(self.batch[0])
        self.loss = self.loss_func(self.preds,self.batch[1])
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        
    def one_epoch(self, train):
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        try: 
            self.callback('before_epoch')
            for self.iter, self.batch in enumerate(self.dl):
                try:
                    self.callback('before_batch')
                    self.one_batch()
                    self.callback('after_batch')
                except CancelBatchException: pass
            self.callback('after_epoch')
        except CancelEpochException: pass
    
    def fit(self, n_epochs):
        self.n_epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        try: 
            self.callback('before_fit')
            for self.epoch in self.epochs:
                self.one_epoch(True)
                self.one_epoch(False)
            self.callback('after_fit')
        except CancelFitException: pass
    
    def callback(self, method_nm): run_cbs(self.cbs, method_nm)

In [102]:
m,nh = 28*28,50
def get_model(): return nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [103]:
model = get_model()
learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=[CompletionCB()])
learn.fit(1)

Completed 69 batches


In [109]:
class SingleBatchCB(Callback):
    order = 1
    def after_batch(self): raise CancelEpochException()

In [110]:
learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=[SingleBatchCB(),CompletionCB()])
learn.fit(1)

Completed 2 batches


## Metrics

In [50]:
class Metric:
    def __init__(self): self.reset()
    def reset(self): self.vals, self.ns = [], []
    def add(self, inp, targ = None, n = 1):
        self.last = self.calc(inp, targ)
        self.vals.append(self.last)
        self.ns.append(n)
    @property
    def value(self):
        ns = tensor(self.ns)
        return (tensor(self.vals)*ns).sum()/ns.sum()
    def calc(self, inps, targs): return inps

In [51]:
class Accuracy(Metric):
    def calc(self, inps, targs): return (inps == targs).float().mean()

In [52]:
acc = Accuracy()
acc.add(tensor([0, 1, 2, 0, 1, 2]), tensor([0, 1, 1, 2, 1, 0]))
acc.add(tensor([1, 1, 2, 0, 1]), tensor([0, 1, 1, 2, 1]))
acc.value

tensor(0.45)

In [53]:
loss = Metric()
loss.add(0.6, n=32)
loss.add(0.9, n=2)
loss.value, round((0.6*32+0.9*2)/(32+2), 2)

(tensor(0.62), 0.62)

# Some Callbacks

In [ ]:
class DeviceCB(Callback):
    def __init__(self, device=def_device): fc.store_attr()
    def before_fit(self): self.learn.model.to(self.device)
    def before_batch(self): self.learn.batch = to_device(self.learn.batch, device = self.device)

## Callbacks Learner

In [54]:
def identity(*args):
    if not args: return
    x, *args = args
    return (x,) + args if args else x

In [55]:
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

In [58]:
class with_cbs:
    def __init__(self, nm): self.nm = nm
    def __call__(self, f):
        def _f(o, *args, **kwargs):
            try:
                o.callback(f'before_{self.nm}')
                f(o, *args, **kwargs)
                o.callback(f'after_{self.nm}')
            except globals()[f'Cancel{self.nm.title()}Exception']: pass
        return _f

In [61]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, cbs, opt_func = optim.SGD): 
        fc.store_attr()
        for cb in cbs: cb.learn = self
    
    @with_cbs('batch')
    def one_batch(self):
        self.predict()
        self.get_loss()
        if self.model.training:
            self.backward()
            self.step()
            self.zero_grad()
            
    def one_epoch(self, train):
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        self._one_epoch()
    
    @with_cbs('epoch')
    def _one_epoch(self):
        for self.iter, self.batch in enumerate(self.dl): self.one_batch()
        
    def fit(self, n_epochs):
        self.n_epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.fit()
        
    @with_cbs('fit')
    def _fit(self):
        for self.epoch in self.epochs:
            self.one_epoch(True)
            self.one_epoch(False)
            
    def __getattr__(self, name):
        if name in ('predict', 'get_loss', 'backward', 'step', 'zero_grad'): return partial(self.callback, name)
        raise AttributeError(name)
    
    def callback(self, method_nm):
        for cb in sorted(self.cbs, key=attrgetter('order')): getattr(cb, method_nm, identity)()

In [62]:
class Callback(): order= 0

In [63]:
class TrainCB(Callback):
    def predict(self): self.learn.preds = self.learn.model(self.learn.batch[0])
    def get_loss(self): self.learn.loss = self.learn.loss_func(self.learn.preds, self.learn.batch[1])
    def backward(self): self.learn.loss.backward()
    def step(self): self.learn.opt.step()
    def zero_grad(self): self.learn.opt.zero_grad()

In [64]:
class DeviceCB(Callback):
    def before_fit(self): self.learn.model.to(def_device)
    def before_batch(self): self.learn.batch = to_device(self.learn.batch)

In [ ]:
class MetricsCB(Callback):
    def __init__(self, metric=None):
        self.loss_m = Metric()
        self.all = [self.loss_m]
        if metric:
            self.metric = metric
            self.all.append(metric)
            
    def log(self, *s): print(self.learn.epoch, self.learn.model.training, *s)
    def before_fit(self): self.learn.metrics = self
    
    def before_epoch(self): [o.reset() for o in self.all]
    def after_epoch(self): self.log(*[f'{o.value:.3f}' for o in self.all])
    
    def after_batch(self):
        if hasattr(self, 'metric'):
            self.metric.add(self.learn.preds.argmax(dim=1), self.learn.batch[1])
        self.loss_m.add(self.learn.loss, n=len(self.learn.batch[0]))